In [1]:
import pandas as pd

In [2]:
def preprocess(data):
    headlines = data[data.keywords_value=='United States Politics and Government'].headline
    headlines = headlines.str.replace("(<br/>)", "")
    headlines = headlines.str.replace('(<a).*(>).*(</a>)', '')
    headlines = headlines.str.replace('(&amp)', '')
    headlines = headlines.str.replace('(&gt)', '')
    headlines = headlines.str.replace('(&lt)', '')
    headlines = headlines.str.replace('(\xa0)', ' ')  
    return headlines

In [3]:
#metadata.sectionName.value_counts()

In [4]:
df = pd.read_csv('./data/headline.csv')
metadata = df.dropna()
data = preprocess(metadata)

In [5]:
df.isnull().sum()

id                  0
headline          159
pub_date            0
month               0
keywords_name       0
keywords_value      0
dtype: int64

In [6]:
import spacy
import markovify
nlp = spacy.load(r'C:\Users\hyh6hhy\Anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.0.0')

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [7]:
import time
start_time = time.time()
headlines_generator_POSified = POSifiedText(data, state_size = 2)
print("Run time for training the generator : {} seconds".format(round(time.time()-start_time, 2)))

Run time for training the generator : 10.43 seconds


In [8]:
def generate_headlines(generator, number=10, short=False):
    count = 0
    while count < number:
        if short:
            headline = generator.make_short_sentence(140)
        else:
            headline = generator.make_sentence()
        if headline:
            count += 1
            #print("Headline {}".format(count))
            headlines.append(headline)
            #print()
    return headlines

In [11]:

import texttable as tt
headlines = []
headlines = generate_headlines(headlines_generator_POSified)
keywords = 'United States Politics and Government'
print("Target Keywords : "+keywords)
tab = tt.Texttable()
headings = ['Headlines']
tab.header(headings)
headlines = headlines
for row in zip(headlines):
    tab.add_row(row)
s = tab.draw()
print(s)


Target Keywords : United States Politics and Government
+------------------------------------------------------------------------------+
|                                  Headlines                                   |
+==============================================================================+
| On Politics  Trump Moves Toward Border Deal Is Reached ‘ in Principle’       |
+------------------------------------------------------------------------------+
| Hillary Clinton Says She Is Not Enough , Anita Hill Says                     |
+------------------------------------------------------------------------------+
| As Government Reopens , the College Years                                    |
+------------------------------------------------------------------------------+
| Pressed by Climate Activists , Senate Foreign Relations Committee Goes Quiet |
+------------------------------------------------------------------------------+
| Barr Says He ’ll Release Mueller Report Head - On  